In [13]:


import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
from pathlib import Path
from typing import Union
import statistics

from tmc_summarizer.data_model import TMC_File, geocode_tmc
from tmc_summarizer.helpers import zip_files


def files_to_process(folder: Path) -> list:
    """Make a list of files to process. File names must meet
    the following criteria:
        - file ends in ``.xls``
        - filename has at least 1 underscore
        - text before the first underscore can be converted to an integer

    :param folder: folder where files are stored
    :type folder: Path
    :return: list of files that meet criteria
    :rtype: list
    """

    # Get a list of all .xls files in the folder
    files = list(folder.glob('**/*.xls'))

    # Remove any files that don't have proper naming conventions
    for f in files:

        # Make sure there is at least 1 underscore
        if "_" not in str(f.name):
            print(f"No underscores, skipping {f.name}")
            files.remove(f)

        # Make sure that the Location ID is an integer
        parts = str(f.name).split("_")

        try:
            _ = int(parts[0])

        except ValueError:
            print(f"Bad Location ID, skipping {f.name}")
            files.remove(f)

    return files



In [14]:

def write_summary_file(input_folder: Union[Path, str],
                       output_folder: Union[Path, str] = None,
                       geocode_helper: str = None) -> Path:
    """
    Create a new ``.xlsx`` summary file.

    This file has two tabs:
        - ``Summary`` contains a single line-item for each TMC
        - ``Detail`` has 4 line-items per TMC:
            - AM Peak Hour Total
            - AM Peak Hour Percent Heavy Vehicles
            - PM Peak Hour Total
            - PM Peak Hour Percent Heavy Vehicles

    TODO: review this format. Maybe 4 tabs instead of  4 rows?

    Outputs a ZIP file with the Excel file and optional geojson file.

    :param input_folder: folder where TMC data is stored
    :type input_folder: Path
    :param output_folder: folder where output ``.xlsx`` file will be stored
    :type output_folder: Path, optional
    :param geocode_helper: text that gets appended to the location
                           name to assist with geocoding precision.
    :type geocode_helper: str, optional but HIGHLY recommended!
    :return: filepath of the new summary ZIP file
    :rtype: Path
    """
    start_time = datetime.now()

    metadata = []
    detailed_data = []

    #these two lists exist to add the peak hours, in seconds, so they can be averaged for the network later
    am_peak_hour_list = [] 
    pm_peak_hour_list = [] 

    input_folder = Path(input_folder)

    # Use the specified output folder
    if output_folder:
        output_folder = Path(output_folder)
    # If none is specified, write to the input folder
    else:
        output_folder = Path(input_folder)

    now_txt_1 = start_time.strftime("%Y-%m-%d %H-%M-%S")
    now_txt_2 = start_time.strftime("%Y_%m_%d_%H_%M_%S")

    output_xlsx_filepath = output_folder / ("TMC Summary " + now_txt_1 + ".xlsx")
    output_geojson_filepath = output_folder / ("tmc_locations_" + now_txt_2 + ".geojson")
    output_zip_file = output_folder / ("tmc_summary_" + now_txt_2 + ".zip")

    all_tmcs = []

    # Extract dataframes from each file, put into appropriate list
    for file in files_to_process(input_folder):
        tmc = TMC_File(file)

        # Single-row metadata DF
        metadata.append(tmc.df_meta)

        # For each cut listed below, get single-row DF
        # -> (am_total, am_heavy_pct, pm_total, pm_heavy_pct)

        for timeperiod in ["am", "pm"]:
            meta_data_peak = list(tmc.df_meta.loc[:,f"{timeperiod}_peak_raw"])
            time = meta_data_peak[0][0].to_pydatetime()
            seconds = (time.hour * 60 + time.minute) * 60 + time.second
            if timeperiod == "am":
                am_peak_hour_list.append(seconds)
            elif timeperiod == "pm":
                pm_peak_hour_list.append(seconds)
            else:
                print("Not a valid time period")
            
            #insert data for total and heavy percentages
            for dtype in ["total", "heavy_pct"]:
                identifier = f"{timeperiod}_{dtype}"

                # Grab the appropriate dataframe
                df = tmc.peak_data[identifier]

                # Insert data into extra columns up front
                df.insert(0, "time", tmc.meta[f"{timeperiod}_peak"])
                df.insert(0, "period", timeperiod)
                df.insert(0, "dtype", dtype)
                df.insert(0, "location_id", tmc.location_id)
                df.insert(0, "location_name", tmc.meta["location_name"])

                detailed_data.append(df)

        all_tmcs.append(tmc)

    # Merge each list of dataframes into its own combined dataframe
    df_meta = pd.concat(metadata)
    df_detail = pd.concat(detailed_data)

    am_peak_hr_seconds = statistics.median(am_peak_hour_list)
    am_network_peak_hour = str(timedelta(seconds=am_peak_hr_seconds))
    pm_peak_hr_seconds = statistics.median(pm_peak_hour_list)
    pm_network_peak_hour = str(timedelta(seconds=pm_peak_hr_seconds))
    df_meta = df_meta.drop(columns=['am_peak_raw','pm_peak_raw'])
    df_meta.insert(4,"pm_network_peak", pm_network_peak_hour)
    df_meta.insert(4,"am_network_peak", am_network_peak_hour)




In [15]:
write_summary_file('/Volumes/GoogleDrive/My Drive/tmc_copy')



Reading 4_Woodside_Ave.xls
Reading 3_Bradford_Ave.xls
Reading 2_Reeves_Ave.xls
Reading 6_Gropp_Ave_Thursday.xls
Reading 5_White_Horse_Ave.xls
Reading 1_Central_Ave.xls


,location_id,location_name,date,time,am_network_peak,pm_network_peak,am_peak,pm_peak,leg_nb,leg_sb,leg_eb,leg_wb,filepath
0,4,162254 - CR 620 Arena Dr & Woodside Ave,2022-03-24,06:00:00 to 19:00:00,7:30:00,15:30:00,08:00 to 09:00,15:30 to 16:30,Woodside Ave,Elementary School Driveway,CR 620 Arena Dr,CR 620 Arena Dr,/Volumes/GoogleDrive/My Drive/tmc_copy/4_Woods...
0,3,162253 - CR 620 Arena Dr & Bradford Ave,2022-03-24,06:00:00 to 19:00:00,7:30:00,15:30:00,07:30 to 08:30,15:30 to 16:30,Bradford Ave,Bradford Ave,CR 620 Arena Dr,CR 620 Arena Dr,/Volumes/GoogleDrive/My Drive/tmc_copy/3_Bradf...
0,2,162252 - CR 620 Arena Dr & Reeves Ave,2022-03-24,06:00:00 to 19:00:00,7:30:00,15:30:00,07:15 to 08:15,15:30 to 16:30,Reeves Ave,Reeves Ave,CR 620 Arena Dr,CR 620 Arena Dr,/Volumes/GoogleDrive/My Drive/tmc_copy/2_Reeve...
0,6,162256 - CR 620 Arena Dr & Gropp Ave Thursday,2022-03-24,06:00:00 to 19:00:00,7:30:00,15:30:00,07:30 to 08:30,15:00 to 16:00,Gropp Ave,GroppAve,CR 620 Arena Dr,CR 620 Arena Dr,/Volumes/GoogleDrive/My Drive/tmc_copy/6_Gropp...
0,5,162255 - CR 620 Arena Dr & White Horse Ave,2022-03-24,06:00:00 to 19:00:00,7:30:00,15:30:00,07:30 to 08:30,16:30 to 17:30,White Horse Ave,White Horse Ave,CR 620 Arena Dr,CR 620 Arena Dr,/Volumes/GoogleDrive/My Drive/tmc_copy/5_White...
